# Librerias

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import signal
import os
%matplotlib widget

# Declaracion de constantes 

In [2]:
F_MIN = 5
F_MAX = 40
F_MUESTREO = 128
RUTA_DATOS_PROCESADOS = '../DatosProcesados/Frecuencia'
CARPETAS = ['Rojo', 'Azul', 'Verde', 'Morado', 'Baseline']

# Declaracion de Funciones 

In [ ]:
def plot_datos(datos):
    
    plt.figure()
    plt.plot(datos[:, 0], label = 'EEG.O1')
    plt.plot(datos[:, 1], label = 'EEG.O2')
    plt.legend()
    plt.title('Raw Data')
    plt.xlabel('Tiempo')
    plt.ylabel('uV')
    
def plot_psd_semilog(datos):
    
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.semilogy(f1, psd_O1, label = 'EEG.O1')
    plt.semilogy(f1, psd_O2, label = 'EEG.O2')
    plt.ylim([1e-7, 1e2])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.legend()

    
def plot_psd(datos):
    
    plt.figure()
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.plot(f1, psd_O1, label = 'EEG.O1')
    plt.plot(f1, psd_O2, label = 'EEG.O2')
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD')
    plt.legend()
    print(psd_O1.mean())
    print(psd_O2.mean())
    print(max(psd_O1))
    print(max(psd_O2))

def filtro_pasa_banda(datos, f_min, f_max, f_muestreo, orden=5):
    nyq = 0.5 * f_muestreo
    f_min = f_min / nyq
    f_max = f_max / nyq
    b, a = signal.butter(orden, [f_min, f_max], btype='band')
    print(b, a)
    y = signal.lfilter(b, a, datos)
    return y



## Creacion de Archivos Transformada de Hilbert 

In [ ]:
from scipy import fftpack 

for carpeta in CARPETAS:
    archivos = [e for e in os.listdir(RUTA_DATOS_PROCESADOS+'/'+carpeta) if e.endswith('.csv') ]
    
    for indice, archivo in enumerate(archivos):
        ruta_origen = f'{RUTA_DATOS_PROCESADOS}/{carpeta}/{archivo}'
        df = pd.read_csv(ruta_origen)
        etiqueta = df[['MarkerValueInt']]
        datos = df[['EEG.O1', 'EEG.O2']].to_numpy()
        o1 = fftpack.hilbert(datos[:, 0])
        o1 = pd.DataFrame(data=o1, columns=["EEG.O1"])
        o2 = fftpack.hilbert(datos[:, 1])
        o2 = pd.DataFrame(data=o2, columns=["EEG.O2"])
        archivo = pd.concat([o1, o2, etiqueta], axis=1).to_csv(f'{RUTA_DATOS_PROCESADOS}/{carpeta}/Hilbert/hilbert{indice}.csv', index = False)

In [ ]:
pwd

In [ ]:
!rm ../DatosProcesados/Frecuencia/Baseline/hilbert*


In [ ]:
features1 = {'Decil 1': [], 'Decil 2': [], 'Decil 3':[], 'Decil 4': [], 'Decil 5': [], 'Decil 6': [], 'Decil 7': [],'Decil 8': [], 'Decil 9': []}
features2 = {'Decil 1': [], 'Decil 2': [], 'Decil 3':[], 'Decil 4': [], 'Decil 5': [], 'Decil 6': [], 'Decil 7': [],'Decil 8': [], 'Decil 9': []}
for carpeta in CARPETAS:
    archivos = [e for e in os.listdir(RUTA_DATOS_PROCESADOS+'/'+carpeta+'/Hilbert') if e.endswith('.csv') ]
    
    for indice, archivo in enumerate(archivos):
        ruta_origen = f'{RUTA_DATOS_PROCESADOS}/{carpeta}/Hilbert/{archivo}'
        df = pd.read_csv(ruta_origen)
    
        features1['Decil 1'].append(df.quantile(.1, axis = 0)[0])
        features1['Decil 2'].append(df.quantile(.2, axis = 0)[0])
        features1['Decil 3'].append(df.quantile(.3, axis = 0)[0])
        features1['Decil 4'].append(df.quantile(.4, axis = 0)[0])
        features1['Decil 5'].append(df.quantile(.5, axis = 0)[0])
        features1['Decil 6'].append(df.quantile(.6, axis = 0)[0])
        features1['Decil 7'].append(df.quantile(.7, axis = 0)[0])
        features1['Decil 8'].append(df.quantile(.8, axis = 0)[0])
        features1['Decil 9'].append(df.quantile(.9, axis = 0)[0])
        
    
        features2['Decil 1'].append(df.quantile(.1, axis = 0)[0])
        features2['Decil 2'].append(df.quantile(.2, axis = 0)[0])
        features2['Decil 3'].append(df.quantile(.3, axis = 0)[0])
        features2['Decil 4'].append(df.quantile(.4, axis = 0)[0])
        features2['Decil 5'].append(df.quantile(.5, axis = 0)[0])
        features2['Decil 6'].append(df.quantile(.6, axis = 0)[0])
        features2['Decil 7'].append(df.quantile(.7, axis = 0)[0])
        features2['Decil 8'].append(df.quantile(.8, axis = 0)[0])
        features2['Decil 9'].append(df.quantile(.9, axis = 0)[0])
        


In [ ]:
O1 = pd.DataFrame(features1)
O2 = pd.DataFrame(features2)
ruta_origen = f'{RUTA_DATOS_PROCESADOS}/Features/'
df1 = pd.read_csv(ruta_origen+'Occipital1.csv')
df2 = pd.read_csv(ruta_origen+'Occipital2.csv')
pd.concat([df1, O1], axis = 1).to_csv('../Features/Occipital1.csv', index = False)
pd.concat([df2, O2], axis = 1).to_csv('../Features/Occipital2.csv', index = False)

In [ ]:
features1 = {'AH0': [], 'AH1': [],'AH2': [],'AH3': [],'AH4': [],'AH5': [],'AH6': [],'AH7': [],'AH8': [], 'AH9': []}
features2 = {'AH0': [], 'AH1': [],'AH2': [],'AH3': [],'AH4': [],'AH5': [],'AH6': [],'AH7': [],'AH8': [], 'AH9': []}
for carpeta in CARPETAS:
    archivos = [e for e in os.listdir(RUTA_DATOS_PROCESADOS+'/'+carpeta+'/Hilbert') if e.endswith('.csv') ]
    
    for indice, archivo in enumerate(archivos):
        ruta_origen = f'{RUTA_DATOS_PROCESADOS}/{carpeta}/Hilbert/{archivo}'
        df = pd.read_csv(ruta_origen)
        a, b = np.histogram(df['EEG.O1'], density=True)
        
        features1['AH0'].append(a[0])
        features1['AH1'].append(a[1])
        features1['AH2'].append(a[2])
        features1['AH3'].append(a[3])
        features1['AH4'].append(a[4])
        features1['AH5'].append(a[5])
        features1['AH6'].append(a[6])
        features1['AH7'].append(a[7])
        features1['AH8'].append(a[8])
        features1['AH9'].append(a[9])
        
        features2['AH0'].append(a[0])
        features2['AH1'].append(a[1])
        features2['AH2'].append(a[2])
        features2['AH3'].append(a[3])
        features2['AH4'].append(a[4])
        features2['AH5'].append(a[5])
        features2['AH6'].append(a[6])
        features2['AH7'].append(a[7])
        features2['AH8'].append(a[8])
        features2['AH9'].append(a[9])
        

In [ ]:
O1 = pd.DataFrame(features1)
O2 = pd.DataFrame(features2)
ruta_origen = f'../Features/'
df1 = pd.read_csv(ruta_origen+'Occipital1.csv')
df2 = pd.read_csv(ruta_origen+'Occipital2.csv')
pd.concat([df1, O1], axis = 1).to_csv('../Features/Occipital1.csv', index = False)
pd.concat([df2, O2], axis = 1).to_csv('../Features/Occipital2.csv', index = False)

In [ ]:
import sklearn


In [5]:
from sklearn import preprocessing
ruta_origen = f'../Features/'
df = pd.read_csv(ruta_origen+'Occipital1copy.csv')
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [6]:
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.000000,0.0,0.000061,0.389937,0.138870,0.166316,0.000071,0.805263,0.340909,0.163934,...,0.038857,0.010296,0.079134,0.111641,0.219701,0.272601,0.252432,0.108358,0.018737,0.012804
1,0.000559,0.0,0.000176,0.314465,0.071110,0.187334,0.000192,0.594737,0.590909,0.344262,...,0.230581,0.332399,0.435175,0.516469,0.474241,0.442129,0.339048,0.134555,0.081436,0.129797
2,0.001117,0.0,0.000208,0.251572,0.087589,0.131676,0.000278,0.636842,0.583333,0.516393,...,0.109861,0.150035,0.252999,0.341790,0.324868,0.375346,0.428995,0.265392,0.234048,0.163280
3,0.001676,0.0,0.000086,0.452830,0.201387,0.223347,0.000098,0.621053,0.666667,0.319672,...,0.065783,0.064607,0.141881,0.192314,0.365928,0.638621,0.653022,0.503979,0.329227,0.246233
4,0.002235,0.0,0.001483,0.314465,0.168038,0.098098,0.001384,0.310526,0.409091,0.483607,...,0.068973,0.094762,0.200673,0.333888,0.523769,0.723891,0.541909,0.335245,0.147825,0.068583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,0.997765,1.0,0.000623,0.490566,0.235414,0.223347,0.000579,0.384211,0.590909,0.483607,...,0.009585,0.000000,0.000000,0.084450,0.325104,0.304573,0.053827,0.012807,0.011295,0.004971
1787,0.998324,1.0,0.000129,0.238994,0.166888,0.221263,0.000151,0.789474,0.727273,0.262295,...,0.295704,0.279446,0.369783,0.304696,0.311523,0.183103,0.060951,0.017744,0.019561,0.017936
1788,0.998883,1.0,0.000321,0.327044,0.421665,0.276210,0.000218,0.668421,0.643939,0.377049,...,0.162727,0.341741,0.507273,0.403672,0.499802,0.540169,0.316666,0.125377,0.046070,0.048005
1789,0.999441,1.0,0.000106,0.452830,0.188217,0.178246,0.000091,0.489474,0.515152,0.163934,...,0.147478,0.080354,0.359227,0.463730,0.526545,0.536303,0.264219,0.049747,0.073120,0.028910


In [4]:
import seaborn as sn
df = pd.read_csv('../Features/Occipital1copy.csv')
corrMatrix = df.corr()
sn.heatmap(corrMatrix)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [ ]:
corrMatrix